## Integrate Newton's Equation of Motion

This example uses the Velocity-Verlet algorithm to integrate the set of equations

$ m \ddot{x} = F(x), F(x) = - dV(x)/dx $

In [1]:
mutable struct Atom
    pos::Vector{Float64}
    vel::Vector{Float64}
    acc::Vector{Float64}
    μ::Float64
    σ::Float64
    ϵ::Float64
end

In [ ]:
# not used
struct Particles
    atom::Vector{Atom}
end

In [ ]:
# not used
function dist(particles::Particles)
    print(particles.atom[1].pos - particles.atom[2].pos)
end

function dist(particles::Particles, particles2::PArticles)
dist(particles)

In [ ]:
# see below for use
num_atoms = 2
particles_generator = []
for n=1:num_atoms
    pos = [n, 0, 0]
    atom = Atom(pos, zeros(3), zeros(3), 1.0, 1.0, 1.0)
    push!(particles_generator, atom)
end

#particles = Particles(particles_generator)
particles = particles_generator
length(particles_generator)

In [2]:
function force(particles, box)
    num_atoms = length(particles)
    f_partial = zeros(num_atoms, num_atoms, 3)  
    f_total = zeros(num_atoms, 3)  
    
    for m in 1:num_atoms, n in (m+1):num_atoms
        dr = particles[m].pos - particles[n].pos
        dr += - round.(dr./box, digits=0) .* box # PBC
        r2 = sum(dr.*dr) / particles[n].σ^2
        r4 = r2 * r2
        r8 = r4 * r4
        r14 = r8 * r4 * r2
        f_partial[m, n, :] = - particles[n].ϵ * (6.0/r8 - 12/r14) * dr
        f_partial[n, m, :] = - f_partial[m, n, :]
    end
   
    f_total1 = sum(f_partial, dims=2)
    f_total1 = dropdims(f_total1, dims=2)
    return f_total1
end

force (generic function with 1 method)

In [3]:
function potential_energy(particles, box)
    num_atoms = length(particles)
    e_total = 0
    for m in 1:num_atoms, n in (m+1):num_atoms
        pos_m = particles[m].pos
        pos_n = particles[n].pos
        dr = pos_m - pos_n - round.((pos_m - pos_n)./ box, digits=0) .* box # PBC
        r2 = sum(dr.*dr) / particles[n].σ^2
        r6 = r2^3
        r12 = r6^2
        e_pair = particles[n].ϵ *(1.0/r12 - 1.0/r6)
        e_total += e_pair
    end
    return e_total
end     

potential_energy (generic function with 1 method)

In [4]:
function kinetic_energy(particles)::Float64
    e_total = 0.0
    for n in 1:length(particles)
        vsq = sum(particles[n].vel .* particles[n].vel)
        e_total += 0.5 * particles[n].μ * vsq 
    end
    return e_total
end

kinetic_energy (generic function with 1 method)

In [5]:
function velocity_verlet(particles, dt)
    num_atoms = length(particles)
    dt2 = dt/2
    dtsq2 = dt^2 / 2
    for n in 1:num_atoms
        particles[n].pos += dt*particles[n].vel + dtsq2*particles[n].acc
        particles[n].vel += dt2*particles[n].acc
    end
    force_update = force(particles, box)
    for n in 1:num_atoms
        particles[n].acc = force_update[n, :] / particles[n].μ
        particles[n].vel += dt2*particles[n].acc
    end
    return particles
end

velocity_verlet (generic function with 1 method)

In [6]:
box = [10, 10, 10]
num_atoms = 2
particles_generator = []
for n=1:num_atoms
    pos = [n*1.04, 0, 0]
    atom = Atom(pos, zeros(3), zeros(3), 1.0, 1.0, 1.0)
    push!(particles_generator, atom)
end

#particles = Particles(particles_generator)
particles = particles_generator
length(particles_generator)

2

In [ ]:
e_e_pot = potential_energy(particles, box)
e_kin = kinetic_energy(particles)
e_tot = e_kin + e_pot
println("initial energy: total $e_tot, potential $e_pot, kinetic $e_kin")
r_list = zeros(0)
v_list = zeros(0)
a_list = zeros(0); t_list = zeros(0)
dt = 0.01
dt_steps = 2000
intermediate = ceil(dt_steps / 00)

for t in 1:dt_steps
    particles = velocity_verlet(particles, dt)
    
    if t % intermediate == 0
        e_pot = round(potential_energy(particles, box), digits = 7)
        e_kin = round(kinetic_energy(particles), digits = 7)
        e_tot = round(e_kin + e_pot, digits = 7)
        println("energy after $t steps: total $e_tot, potential $e_pot, kinetic $e_kin")
        dr = particles[1].pos - particles[2].pos
        dist = sqrt(sum(dr.*dr))
        dvx = particles[1].vel[1] - particles[2].vel[1]
        dax = particles[1].acc[1] - particles[2].acc[1]
        append!(r_list, dist)
        append!(v_list, dvx)
        append!(a_list, dax)
        append!(t_list, t)
    end
end

pot = potential_energy(particles, box)
e_kin = kinetic_energy(particles)
e_tot = e_kin + e_pot
println("initial energy: total $e_tot, potential $e_pot, kinetic $e_kin")
r_list = zeros(0)
v_list = zeros(0)
a_list = zeros(0); t_list = zeros(0)
dt = 0.01
dt_steps = 2000
intermediate = ceil(dt_steps / 00)

for t in 1:dt_steps
    particles = velocity_verlet(particles, dt)
    
    if t % intermediate == 0
        e_pot = round(potential_energy(particles, box), digits = 7)
        e_kin = round(kinetic_energy(particles), digits = 7)
        e_tot = round(e_kin + e_pot, digits = 7)
        println("energy after $t steps: total $e_tot, potential $e_pot, kinetic $e_kin")
        dr = particles[1].pos - particles[2].pos
        dist = sqrt(sum(dr.*dr))
        dvx = particles[1].vel[1] - particles[2].vel[1]
        dax = particles[1].acc[1] - particles[2].acc[1]
        append!(r_list, dist)
        append!(v_list, dvx)
        append!(a_list, dax)
        append!(t_list, t)
    end
end

e_pot = potential_energy(particles, box)
e_kin = kinetic_energy(particles)
e_tot = e_kin + e_pot
println("initial energy: total $e_tot, potential $e_pot, kinetic $e_kin")
r_list = zeros(0)
v_list = zeros(0)
a_list = zeros(0); t_list = zeros(0)
dt = 0.01
dt_steps = 2000
intermediate = ceil(dt_steps / 00)

for t in 1:dt_steps
    particles = velocity_verlet(particles, dt)
    
    if t % intermediate == 0
        e_pot = round(potential_energy(particles, box), digits = 7)
        e_kin = round(kinetic_energy(particles), digits = 7)
        e_tot = round(e_kin + e_pot, digits = 7)
        println("energy after $t steps: total $e_tot, potential $e_pot, kinetic $e_kin")
        dr = particles[1].pos - particles[2].pos
        dist = sqrt(sum(dr.*dr))
        dvx = particles[1].vel[1] - particles[2].vel[1]
        dax = particles[1].acc[1] - particles[2].acc[1]
        append!(r_list, dist)
        append!(v_list, dvx)
        append!(a_list, dax)
        append!(t_list, t)
    end
end



In [7]:
e_pot = potential_energy(particles, box)
e_kin = kinetic_energy(particles)
e_tot = e_kin + e_pot
println("initial energy: total $e_tot, potential $e_pot, kinetic $e_kin")
r_list = zeros(0)
v_list = zeros(0)
a_list = zeros(0); t_list = zeros(0)
dt = 0.01
dt_steps = 2000
intermediate = ceil(dt_steps / 00)

for t in 1:dt_steps
    particles = velocity_verlet(particles, dt)
    
    if t % intermediate == 0
        e_pot = round(potential_energy(particles, box), digits = 7)
        e_kin = round(kinetic_energy(particles), digits = 7)
        e_tot = round(e_kin + e_pot, digits = 7)
        println("energy after $t steps: total $e_tot, potential $e_pot, kinetic $e_kin")
        dr = particles[1].pos - particles[2].pos
        dist = sqrt(sum(dr.*dr))
        dvx = particles[1].vel[1] - particles[2].vel[1]
        dax = particles[1].acc[1] - particles[2].acc[1]
        append!(r_list, dist)
        append!(v_list, dvx)
        append!(a_list, dax)
        append!(t_list, t)
    end
end


initial energy: total -0.1657174761500806, potential -0.1657174761500806, kinetic 0.0


In [ ]:
using Plots
gr()

plot(t_list, v_list, 
    xlabel="t", 
    ylabel="dist",
    title="Distance",
    legend = false)